# Query the EFD for ATDome commands

Craig Lage - 18-Oct-24

In [ ]:
import sys, time, os, asyncio, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient

In [ ]:
client = EfdClient('summit_efd') # 'usdf_efd' at USDF

# Queries

In [ ]:
start = Time("2024-10-18 10:00:00Z", scale='utc')
end = Time("2024-10-18 15:00:00Z", scale='utc')
commands = []
topics = await client.get_topics()
for topic in topics:
    if 'ATDome' in topic and 'command' in topic and 'Trajectory' not in topic:
        commands.append(topic)
        
file = open("/home/cslage/DATA/ATDome_Commands_18Oct24", "w")
file.write(f"ATDome commands from {start.isot} to {end.isot} \n\n")
for command in commands:
    data = await client.select_time_series(command, ['*'], start, end)
    file.write(f"{command} had {len(data)} commands\n\n")
    for i in range(len(data)):
        if command == 'lsst.sal.ATDome.command_moveAzimuth':
            file.write(f"{data.index[i]}  {data['azimuth'].values[i]:.2f}\n")
        else:
            file.write(f"{data.index[i]}\n")
    file.write("\n")
file.close()